In [1]:
!pip install bitsandbytes
!pip install ctranslate2
!pip install transformers
!pip install --upgrade torchvision
!pip install datasets
!pip install evaluate sacrebleu
!pip install wandb
!pip install --upgrade --upgrade-strategy eager optimum[onnxruntime]
!pip install bitsandbytes-cuda117==0.26.0.post2

In [2]:
import re

from tqdm import tqdm
import torch
from torch.nn.utils.rnn import pad_sequence
from torch import nn
from torch.utils.data import DataLoader

from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer, DataCollatorForSeq2Seq, AutoTokenizer, TrainingArguments, Trainer, PushToHubCallback
import ctranslate2

from torch.optim import AdamW
import evaluate
from tqdm.auto import tqdm
import os
import numpy as np

import wandb


In [3]:
from huggingface_hub import notebook_login

notebook_login() 

In [4]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [40]:
HF_DATASET_NAME = "data354/Koumankan_mt_dyu_fr"
SRC_LANG = "dyu"
TRG_LANG = "fr"
CHARS_TO_REMOVE_REGEX = '[!"&"\(\),-./:;=?+.\n\[\]]'
PREFIX = "translate Dyula to French: " # This command will have to be passed to the model during inference so it knows what to do
MODEL_CHECKPOINT = "facebook/nllb-200-distilled-600M"
HF_USERNAME = "barro"
HF_REPO_NAME = "dyu-fr-nllb-quantized"
LOCAL_SAVE_DIR = "../saved_model"
batch_size = 8 # @param {type: "integer"}
TRAIN_EPOCHS = 5 #@param {type: "integer"}
learning_rate = 2e-5 #@param {type: "number"}
weight_decay=0.01 #@param {type: "number"}

In [1]:
ct2-transformers-converter --model  --quantization int8 --output_dir dyu-fr-nllb-distilled_ctranslate2

SyntaxError: invalid syntax (<ipython-input-1-2c9e78f655e4>, line 1)

In [7]:
ct2-transformers-converter --model "facebook/nllb-200-distilled-600M" --output_dir "nllb-200-distilled-600M/"

SyntaxError: invalid syntax (<ipython-input-7-57dd70ee9e18>, line 1)

In [32]:
# Download data
dataset = load_dataset(HF_DATASET_NAME)
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'translation'],
        num_rows: 8065
    })
    validation: Dataset({
        features: ['ID', 'translation'],
        num_rows: 1471
    })
    test: Dataset({
        features: ['ID', 'translation'],
        num_rows: 1393
    })
})

In [28]:
# Inspect
print(dataset["train"][0])
print(dataset["train"][2])
print(dataset["train"][4])
print(dataset["train"][5])
print(dataset["train"][6])


{'ID': 'ID_18897661270129', 'translation': {'dyu': 'A bi ji min na', 'fr': 'Il boit de l’eau.'}}
{'ID': 'ID_18164131280307', 'translation': {'dyu': 'Mun? Fɛn dɔ.', 'fr': 'Quoi ? Quelque chose.'}}
{'ID': 'ID_18127342282717', 'translation': {'dyu': 'A ale lo bi da bugɔ la!', 'fr': 'Ah ! c’est lui… il sonne…'}}
{'ID': 'ID_18113952728392', 'translation': {'dyu': "E nafa t'a ra.", 'fr': 'Et sans rapport.'}}
{'ID': 'ID_17349291285068', 'translation': {'dyu': 'Sanu baa de bi n bolo', 'fr': "J'ai suffisamment d'or."}}


In [33]:
def preprocess_function(examples):
    inputs = [PREFIX + example[SRC_LANG] for example in examples["translation"]]
    targets = [example[TRG_LANG] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [36]:
# Clean data (lowercase and punctuation)

def remove_special_characters(text):
    text = re.sub(CHARS_TO_REMOVE_REGEX, " ", text.lower())
    return text.strip()

def clean_text(batch):
    # process source text
    batch["translation"][SRC_LANG] = remove_special_characters(batch["translation"][SRC_LANG])
    # process target text
    batch["translation"][TRG_LANG] = remove_special_characters(batch["translation"][TRG_LANG])
    return batch

dataset = dataset.map(clean_text)
dataset

Map:   0%|          | 0/8065 [00:00<?, ? examples/s]

Map:   0%|          | 0/1471 [00:00<?, ? examples/s]

Map:   0%|          | 0/1393 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ID', 'translation'],
        num_rows: 8065
    })
    validation: Dataset({
        features: ['ID', 'translation'],
        num_rows: 1471
    })
    test: Dataset({
        features: ['ID', 'translation'],
        num_rows: 1393
    })
})

In [ ]:
import ctranslate2
import transformers

src_lang = "eng_Latn"
tgt_lang = "fra_Latn"

translator = ctranslate2.Translator("nllb-200-distilled-600M")
tokenizer = transformers.AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang=src_lang)

source = tokenizer.convert_ids_to_tokens(tokenizer.encode("Hello world!"))
target_prefix = [tgt_lang]
results = translator.translate_batch([source], target_prefix=[target_prefix])
target = results[0].hypotheses[0][1:]

print(tokenizer.decode(tokenizer.convert_tokens_to_ids(target)))

In [50]:
import ctranslate2
import transformers

In [52]:
# NLLB 1.3B - CTranslate2 int8
!wget https://pretrained-nmt-models.s3.us-west-2.amazonaws.com/CTranslate2/nllb/nllb-200_1.2B_int8_ct2.zip

SyntaxError: invalid decimal literal (<ipython-input-52-1a8cb9ade2e0>, line 1)

In [45]:
# NLLB 3.3B - CTranslate2 int8
!wget https://pretrained-nmt-models.s3.us-west-2.amazonaws.com/CTranslate2/nllb/nllb-200_3.3B_int8_ct2.zip

--2024-08-11 11:34:20--  https://pretrained-nmt-models.s3.us-west-2.amazonaws.com/CTranslate2/nllb/nllb-200_3.3B_int8_ct2.zip
Resolving pretrained-nmt-models.s3.us-west-2.amazonaws.com (pretrained-nmt-models.s3.us-west-2.amazonaws.com)... 52.218.170.90, 3.5.79.143, 52.92.186.50, ...
Connecting to pretrained-nmt-models.s3.us-west-2.amazonaws.com (pretrained-nmt-models.s3.us-west-2.amazonaws.com)|52.218.170.90|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-08-11 11:34:21 ERROR 403: Forbidden.



In [47]:
!ct2-download --model nllb-200-distilled-600M

/bin/bash: line 1: ct2-download: command not found


In [53]:
!ct2-download --model nllb-200-distilled-600M
!unzip nllb-200-distilled-600M.zip

/bin/bash: line 1: ct2-download: command not found
unzip:  cannot find or open nllb-200-distilled-600M.zip, nllb-200-distilled-600M.zip.zip or nllb-200-distilled-600M.zip.ZIP.


In [48]:
src_lang = "dyu_Latn"
tgt_lang = "fra_Latn"

# # Check GPU availability and set the device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")

# # Load the tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT, src_lang="dyu_Latn")
# model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT).to(device)

translator = ctranslate2.Translator("nllb-200-distilled-600M")
tokenizer = transformers.AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang=src_lang)

# Set the target language for the tokenizer
tokenizer.tgt_lang = "fr_Latn" # Set the target language
# metric = evaluate.load("bleu")

RuntimeError: Unable to open file 'model.bin' in model 'nllb-200-distilled-600M'

In [ ]:
#@title preprocess function
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_labels = labels

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay= weight_decay)

### Preprocess dataset

In [ ]:
#  Preprocess dataset
def preprocess_function(examples):
    inputs = [ex['dyu'] for ex in examples['translation']] # Extract 'dyu' values from each dictionary
    targets = [ex['fr'] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, return_tensors="pt", padding=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, return_tensors="pt", padding=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Train
train_dataset = dataset['train'].map(preprocess_function, batched=True)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Validation
test_dataset = dataset['validation'].map(preprocess_function, batched=True)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


def collate_fn(batch):
    # Collate the data
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['labels'] for item in batch]

    # Pad sequences to the same length
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100) # Ignore padded labels

    input_ids = torch.tensor(input_ids, dtype = torch.long)
    attention_mask = torch.tensor(attention_mask, dtype = torch.long)
    labels = torch.tensor(labels, dtype = torch.long)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels,
    }

Map:   0%|          | 0/8065 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4144: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1471 [00:00<?, ? examples/s]

### Training

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_steps=10_000,
    save_total_limit=2,
    num_train_epochs=TRAIN_EPOCHS,
    push_to_hub=True,
    hub_model_id=f"{HF_USERNAME}/{HF_REPO_NAME}",
    hub_token=""
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=collate_fn,
    optimizers=(AdamW(model.parameters(), lr=learning_rate), None),  # No scheduler needed
    # callbacks=[PushToHubCallback(output_dir="./results")] # Pass output_dir to PushToHubCallback
)

# Train the model
trainer.train()
print("Training complete!")


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: barro (barro-african-masters-of-machine-intelligence). Use `wandb login --relogin` to force relogin


<ipython-input-17-8622f94f7c4b>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids, dtype = torch.long)
<ipython-input-17-8622f94f7c4b>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(attention_mask, dtype = torch.long)
<ipython-input-17-8622f94f7c4b>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype = torch.long)


Epoch,Training Loss,Validation Loss
1,0.515500,0.845851
2,0.483800,0.856957
3,0.458700,0.869734
4,0.421400,0.881280
5,0.409300,0.887363


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}
<ipython-input-17-8622f94f7c4b>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids, dtype = torch.long)
<ipython-input-17-8622f94f7c4b>:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(attention_mask, dtype = torch.long)
<ipython-input-17-8622f94f7c4b>:33: UserWarning: To copy construct

Training complete!


In [ ]:
# Push the final model to the Hugging Face Hub
trainer.push_to_hub()

push_to_hub_callback = PushToHubCallback(
    output_dir=LOCAL_SAVE_DIR,
    hub_model_id=f"{HF_USERNAME}/{HF_REPO_NAME}",
    tokenizer=tokenizer,
    )

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200}


events.out.tfevents.1723105950.1b64924483c1.1522.0:   0%|          | 0.00/8.98k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/barro/dyu-fr-nllb-quantized into local empty directory.


Download file model.safetensors:   0%|          | 8.00k/2.29G [00:00<?, ?B/s]

Download file runs/Aug08_08-32-30_1b64924483c1/events.out.tfevents.1723105950.1b64924483c1.1522.0: 100%|######…

...

Download file sentencepiece.bpe.model:   0%|          | 3.40k/4.63M [00:00<?, ?B/s]

Download file tokenizer.json:   0%|          | 32.0k/16.5M [00:00<?, ?B/s]

Clean file sentencepiece.bpe.model:   0%|          | 1.00k/4.63M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/16.5M [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/2.29G [00:00<?, ?B/s]

In [ ]:
# Save the model
model.save_pretrained("trained_nllb_dyu_fr-0.3.0")
# Save the tokenizer
tokenizer.save_pretrained("token_trained_nllb_dyu_fr-0.3.0")

### First quantization strategy

In [ ]:
from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForSeq2SeqLM

model_name = "barro/dyu-fr-nllb-distilled"
model = ORTModelForSeq2SeqLM.from_pretrained(model_name, export=True)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# model.save_pretrained("dyu-fr-nllb-distilled_onnx")
# tokenizer.save_pretrained("dyu-fr-nllb-distilled_onnx")

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from optimum.onnxruntime import ORTQuantizer, ORTModelForSeq2SeqLM
from optimum.onnxruntime.configuration import AutoQuantizationConfig

KeyboardInterrupt: 

In [ ]:
encoder_quantizer = ORTQuantizer.from_pretrained("dyu-fr-nllb-distilled_onnx", file_name="encoder_model.onnx")

decoder_quantizer = ORTQuantizer.from_pretrained("dyu-fr-nllb-distilled_onnx", file_name="decoder_model.onnx")

decoder_wp_quantizer = ORTQuantizer.from_pretrained("dyu-fr-nllb-distilled_onnx", file_name="decoder_with_past_model.onnx")

quantizer = [encoder_quantizer, decoder_quantizer, decoder_wp_quantizer]

In [ ]:
dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

for q in quantizer:
    q.quantize(save_dir="dyu-fr-nllb-distilled_onnx_quantized",quantization_config=dqconfig)  # doctest: +IGNORE_RESULT

In [ ]:
from optimum.onnxruntime import ORTQuantizer, ORTModelForSeq2SeqLM
from optimum.onnxruntime.configuration import AutoQuantizationConfig


quantizer = ORTQuantizer.from_pretrained("./dyu-fr-nllb-distilled_onnx",file_name="encoder_model.onnx")

dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

model_quantized_path = quantizer.quantize(
    save_dir="barro/dyu-fr-nllb-distilled_quantized",
    quantization_config=dqconfig,
)

In [ ]:
from optimum.onnxruntime import ORTQuantizer, ORTModelForSeq2SeqLM
from optimum.onnxruntime.configuration import AutoQuantizationConfig


quantizer = ORTQuantizer.from_pretrained("./dyu-fr-nllb-distilled_onnx",file_name="decoder_model.onnx")

dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

model_quantized_path = quantizer.quantize(
    save_dir="barro/dyu-fr-nllb-distilled_quantized",
    quantization_config=dqconfig,
)

### Second strategy

In [ ]:
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)
model_4bit = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_CHECKPOINT,
    device_map="auto",
    quantization_config=quantization_config,
)

### Third strategy

### Evaluation

In [ ]:
# Initialize the list to store translations and references
French_translation = []
references = []

for item in dyu_fr["validation"]:
    # Extract the Dyula text and the reference French translation
    dyu_text = item["translation"]["dyu"]
    reference_text = item["translation"]["fr"]

    # Tokenize the input text
    inputs = tokenizer(dyu_text, return_tensors="pt")

    # Move the input tensors to the specified device
    input_ids = inputs["input_ids"].to(device)
    att_masks = inputs["attention_mask"].to(device)

    # # Generate the translation
    translated_tokens = model.generate(
        input_ids=input_ids,
        attention_mask=att_masks,
        forced_bos_token_id=tokenizer.lang_code_to_id["fra_Latn"])

    translation = translate_sentence(dyula_sentence, tgt_lang="fra_Latn")

    # Decode the translated tokens and get the resulting French translation
    translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

    # Append the translation and the reference to the lists
    French_translation.append(translation)
    references.append([reference_text])

# Calculate the BLEU score
bleu = metric.compute(predictions=French_translation, references=references)

# Print the BLEU score
print(f"BLEU score: {bleu}")
